In [14]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from scipy.stats import levene

Load data

In [3]:
data = pd.read_csv("data/engineered_morality.csv")
data.columns

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_1843/1586172768.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/engineered_morality.csv")


Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'created', 'author', 'score',
       'num_comments', 'link', 'cleaned_text', 'word_count', 'type', 'link_id',
       'year', 'month', 'Segment_1', 'emo_pos', 'emo_neg', 'emo_anx',
       'emo_anger', 'emo_sad', 'moral', 'Segment', 'Care_Virtue', 'Care_Vice',
       'Fairness_Virtue', 'Fairness_Vice', 'Loyalty_Virtue', 'Loyalty_Vice',
       'Authority_Virtue', 'Authority_Vice', 'Sanctity_Virtue',
       'Sanctity_Vice', 'Care_total', 'Fairness_total', 'Loyalty_total',
       'Authority_total', 'Sanctity_total', 'Virtue_total', 'Vice_total',
       'Foundations_total_score', 'Subreddit'],
      dtype='object')

### LIWC moral score

Observe means

In [11]:
sub_means = data.groupby('Subreddit')['moral'].mean()
print(sub_means)

Subreddit
homeowners         0.154164
investing          0.191849
selfimprovement    0.320543
Name: moral, dtype: float64


Observe variance of moral scores

In [11]:
sub_variances = data.groupby('Subreddit')['moral'].var()
print(sub_variances)

Subreddit
homeowners         0.241381
investing          0.297560
selfimprovement    0.474709
Name: moral, dtype: float64


Statistically check differences in variances

In [16]:
# Split the data by subreddit
group1 = data[data['Subreddit'] == 'selfimprovement']['moral']
group2 = data[data['Subreddit'] == 'homeowners']['moral']
group3 = data[data['Subreddit'] == 'investing']['moral']

# Levene's test for equal variances
stat, p_value = levene(group1, group2, group3)

print(f"Levene’s test statistic: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

Levene’s test statistic: 11379.1347
p-value: 0.0000


Heterogeneous variances

Create a binary variable: 1 if morality score > 0, else 0. This is to try to model the likelihood of containing *any* moral language.

In [4]:
data['moral_present'] = (data['moral'] > 0).astype(int)

Check binary distribution

In [6]:
# Get value counts 
counts = data['moral_present'].value_counts()

# Convert to percentages
percentages = counts / counts.sum() * 100

summary_df = pd.DataFrame({
    'Count': counts,
    'Percentage': percentages.round(2)
})

print(summary_df)

                 Count  Percentage
moral_present                     
0              1219464       80.53
1               294777       19.47


Check distribution by subreddit

In [8]:
subreddit_percent = data.groupby('Subreddit')['moral_present'].value_counts(normalize=True).unstack().fillna(0) * 100
subreddit_percent = subreddit_percent.round(2)
print(subreddit_percent)

moral_present        0      1
Subreddit                    
homeowners       86.95  13.05
investing        83.78  16.22
selfimprovement  70.95  29.05


Make the subreddit column a categorical variable that will be used as a categorical predictor (X)

In [ ]:
data['Subreddit'] = data['subreddit'].astype('category')

Logistic model --> Is moral language present?

Run the logistic regression

In [12]:
logit_model = smf.glm(
    formula='moral_present ~ C(Subreddit)',
    data=data,
    family=sm.families.Binomial()
).fit(cov_type='HC1')  # HC1 is a covariance estimator that adjusts for 
                          #heteroskedasticity

print(logit_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:          moral_present   No. Observations:              1514241
Model:                            GLM   Df Residuals:                  1514238
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -7.2397e+05
Date:                Fri, 04 Apr 2025   Deviance:                   1.4479e+06
Time:                        21:25:25   Pearson chi2:                 1.51e+06
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02920
Covariance Type:                  HC1                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

Marginal effects

In [21]:
mfx = logit_model.get_margeff()
print(mfx.summary())

         GLM Marginal Effects        
Dep. Variable:          moral_present
Method:                          dydx
At:                           overall
                                     dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
C(Subreddit)[T.investing]           0.0386      0.001     44.861      0.000       0.037       0.040
C(Subreddit)[T.selfimprovement]     0.1525      0.001    197.367      0.000       0.151       0.154


Get probabilities of showing moral language, to make it more intuitive

In [17]:
# Coefficients from the GLM output
intercept = -1.8963  # Homeowners (baseline)
coef_investing = 0.2541
coef_selfimprovement = 1.0033

Compute log-odds for each subreddit

In [18]:
logit_homeowners = intercept 
logit_investing = intercept + coef_investing
logit_selfimprovement = intercept + coef_selfimprovement

print("Log-Odds:")
print(f"Homeowners: {logit_homeowners}")
print(f"Investing: {logit_investing}")
print(f"Selfimprovement: {logit_selfimprovement}")


Log-Odds:
Homeowners: -1.8963
Investing: -1.6422
Selfimprovement: -0.893


Convert log-odds with predicted probabilities

In [20]:
# Logistic function
def logit_to_prob(logit):
    return np.exp(logit) / (1 + np.exp(logit))

# Apply to each group
prob_homeowners = logit_to_prob(logit_homeowners)
prob_investing = logit_to_prob(logit_investing)
prob_selfimprovement = logit_to_prob(logit_selfimprovement)

print("\nPredicted Probabilities:")
print(f"Homeowners: {prob_homeowners:.4f}")
print(f"Investing: {prob_investing:.4f}")
print(f"Selfimprovement: {prob_selfimprovement:.4f}")



Predicted Probabilities:
Homeowners: 0.1305
Investing: 0.1622
Selfimprovement: 0.2905


### Moral Foundations Dictionary Score

Obseve means

In [12]:
sub_means_mfd = data.groupby('Subreddit')['Foundations_total_score'].mean()
print(sub_means_mfd)

Subreddit
homeowners         1.409286
investing          1.582064
selfimprovement    1.916615
Name: Foundations_total_score, dtype: float64


Observe variances

In [13]:
sub_variances_mfd = data.groupby('Subreddit')['Foundations_total_score'].var()
print(sub_variances_mfd)

Subreddit
homeowners         2.551364
investing          3.071992
selfimprovement    3.109352
Name: Foundations_total_score, dtype: float64


Levene tests to check differences in variances

In [17]:
# Split the data by subreddit
group1 = data[data['Subreddit'] == 'selfimprovement']['Foundations_total_score']
group2 = data[data['Subreddit'] == 'homeowners']['Foundations_total_score']
group3 = data[data['Subreddit'] == 'investing']['Foundations_total_score']

# Levene's test for equal variances
stat, p_value = levene(group1, group2, group3)

print(f"Levene’s test statistic: {stat:.4f}")
print(f"p-value: {p_value:.4f}")


Levene’s test statistic: 1371.0129
p-value: 0.0000


Variances are different here too. 

Create a binary variable: 1 if morality score > 0, else 0. This is to try to model the likelihood of containing *any* moral language.

In [18]:
data['moral_present_mfd'] = (data['Foundations_total_score'] > 0).astype(int)

Check binary distribution

In [19]:
# Get value counts 
counts = data['moral_present_mfd'].value_counts()

# Convert to percentages
percentages = counts / counts.sum() * 100

summary_df = pd.DataFrame({
    'Count': counts,
    'Percentage': percentages.round(2)
})

print(summary_df)

                     Count  Percentage
moral_present_mfd                     
1                  1061487        70.1
0                   452754        29.9


Check distribution by subreddit

In [20]:
subreddit_percent = data.groupby('Subreddit')['moral_present_mfd'].value_counts(normalize=True).unstack().fillna(0) * 100
subreddit_percent = subreddit_percent.round(2)
print(subreddit_percent)

moral_present_mfd      0      1
Subreddit                      
homeowners         35.80  64.20
investing          33.22  66.78
selfimprovement    20.76  79.24


Logistic model

In [23]:
logit_model_mfd = smf.glm(
    formula='moral_present_mfd ~ C(Subreddit)',
    data=data,
    family=sm.families.Binomial()
).fit(cov_type='HC1')  # HC1 again for heteroskedasticity

print(logit_model_mfd.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:      moral_present_mfd   No. Observations:              1514241
Model:                            GLM   Df Residuals:                  1514238
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.0751e+05
Date:                Sun, 06 Apr 2025   Deviance:                   1.8150e+06
Time:                        12:04:00   Pearson chi2:                 1.51e+06
No. Iterations:                     5   Pseudo R-squ. (CS):            0.02116
Covariance Type:                  HC1                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

Marginal effects

In [24]:
mfx = logit_model_mfd.get_margeff()
print(mfx.summary())

         GLM Marginal Effects        
Dep. Variable:      moral_present_mfd
Method:                          dydx
At:                           overall
                                     dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
C(Subreddit)[T.investing]           0.0234      0.001     27.182      0.000       0.022       0.025
C(Subreddit)[T.selfimprovement]     0.1551      0.001    171.228      0.000       0.153       0.157


Get probabilities of showing moral language, to make it more intuitive

In [25]:
# Coefficients from the GLM output
intercept = 0.5841 #Homeowners (baseline)
coef_investing = 0.1139
coef_selfimprovement = 0.7556

Compute log-odds for each subreddit

In [26]:
logit_homeowners = intercept 
logit_investing = intercept + coef_investing
logit_selfimprovement = intercept + coef_selfimprovement

print("Log-Odds:")
print(f"Homeowners: {logit_homeowners}")
print(f"Investing: {logit_investing}")
print(f"Selfimprovement: {logit_selfimprovement}")

Log-Odds:
Homeowners: 0.5841
Investing: 0.698
Selfimprovement: 1.3397000000000001


Convert log-odds with predicted probabilities

In [27]:
# Logistic function
def logit_to_prob(logit):
    return np.exp(logit) / (1 + np.exp(logit))

# Apply to each group
prob_homeowners = logit_to_prob(logit_homeowners)
prob_investing = logit_to_prob(logit_investing)
prob_selfimprovement = logit_to_prob(logit_selfimprovement)

print("\nPredicted Probabilities:")
print(f"Homeowners: {prob_homeowners:.4f}")
print(f"Investing: {prob_investing:.4f}")
print(f"Selfimprovement: {prob_selfimprovement:.4f}")


Predicted Probabilities:
Homeowners: 0.6420
Investing: 0.6677
Selfimprovement: 0.7924


## Conclusion

According to both the LIWC's moral dimension and The Moral Foundations dictionary, the subreddit r/selfimprovement has a higher probability of showing moral language.